In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [2]:
df=pd.read_csv('autos.csv',header=0,sep=',',encoding='Latin1')

C:\Users\sanja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,24-03-2016 11:52,Golf_3_1.6,privat,Angebot,480.0,test,NaN,1993.0,manuell,0.0,golf,150000,0.0,benzin,volkswagen,NaN,24-03-2016 00:00,0.0,70435.0,07-04-2016 03:16
1,24-03-2016 10:58,A5_Sportback_2.7_Tdi,privat,Angebot,18300.0,test,coupe,2011.0,manuell,190.0,NaN,125000,5.0,diesel,audi,ja,24-03-2016 00:00,0.0,66954.0,07-04-2016 01:46
2,14-03-2016 12:52,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800.0,test,suv,2004.0,automatik,163.0,grand,125000,8.0,diesel,jeep,NaN,14-03-2016 00:00,0.0,90480.0,05-04-2016 12:47
3,17-03-2016 16:54,GOLF_4_1_4__3TÜRER,privat,Angebot,1500.0,test,kleinwagen,2001.0,manuell,75.0,golf,150000,6.0,benzin,volkswagen,nein,17-03-2016 00:00,0.0,91074.0,17-03-2016 17:40
4,31-03-2016 17:25,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600.0,test,kleinwagen,2008.0,manuell,69.0,fabia,90000,7.0,diesel,skoda,nein,31-03-2016 00:00,0.0,60437.0,06-04-2016 10:17


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371539 entries, 0 to 371538
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dateCrawled          371539 non-null  object 
 1   name                 371539 non-null  object 
 2   seller               371538 non-null  object 
 3   offerType            371538 non-null  object 
 4   price                371538 non-null  float64
 5   abtest               371538 non-null  object 
 6   vehicleType          333669 non-null  object 
 7   yearOfRegistration   371537 non-null  float64
 8   gearbox              351329 non-null  object 
 9   powerPS              371538 non-null  float64
 10  model                351054 non-null  object 
 11  kilometer            371538 non-null  object 
 12  monthOfRegistration  371537 non-null  float64
 13  fuelType             338151 non-null  object 
 14  brand                371537 non-null  object 
 15  notRepairedDamage

In [5]:
print(df['seller'].value_counts())
print(df['offerType'].value_counts())

privat        371534
gewerblich         3
golf               1
Name: seller, dtype: int64
Angebot    371525
Gesuch         12
150000          1
Name: offerType, dtype: int64


In [6]:
df.drop(columns=['seller','offerType','name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'],inplace=True,axis=1)

In [7]:
df.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480.0,NaN,1993.0,manuell,0.0,golf,150000,0.0,benzin,volkswagen,NaN
1,18300.0,coupe,2011.0,manuell,190.0,NaN,125000,5.0,diesel,audi,ja
2,9800.0,suv,2004.0,automatik,163.0,grand,125000,8.0,diesel,jeep,NaN
3,1500.0,kleinwagen,2001.0,manuell,75.0,golf,150000,6.0,benzin,volkswagen,nein
4,3600.0,kleinwagen,2008.0,manuell,69.0,fabia,90000,7.0,diesel,skoda,nein


In [8]:
df.shape

(371539, 11)

In [9]:
df=df[(df['powerPS']>50) & (df['powerPS']<90)]

In [10]:
df=df[(df['yearOfRegistration']>1949) & (df['yearOfRegistration']<2017)]
df=df[(df['price']>=100) & (df['price']<=150000)]

In [11]:
df.shape

(78684, 11)

In [12]:
df.columns

Index(['price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS',
       'model', 'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage'],
      dtype='object')

In [13]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS','model', 'kilometer', 'monthOfRegistration', 'fuelType','brand','notRepairedDamage'])

In [14]:
new_df['gearbox'].replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df['fuelType'].replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df['vehicleType'].replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df['notRepairedDamage'].replace(('ja','nein'),('yes','no'),inplace=True)


In [15]:
new_df.to_csv('autos_preprocessed.csv')

In [16]:
labels=['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']

In [17]:
dicti={}
for feature in labels:
    dicti[feature]=LabelEncoder()
    dicti[feature].fit(new_df[feature])
    temp=dicti[feature].transform(new_df[feature])
    np.save(str('numpy_classes/classes'+feature+'.npy'),dicti[feature].classes_)
    print(feature,':',dicti[feature])
    new_df.loc[:,feature+'_Labels']=pd.Series(temp,index=new_df.index)
labeled=new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration']+[x+'_Labels' for x in labels]]


gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()


In [18]:
labeled.columns

Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_Labels', 'notRepairedDamage_Labels',
       'model_Labels', 'brand_Labels', 'fuelType_Labels',
       'vehicleType_Labels'],
      dtype='object')

In [19]:
X=labeled.iloc[:,1:].values
Y=labeled.iloc[:,0].values

In [20]:
Y=Y.reshape(-1,1)

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=.3)

In [22]:
regressor=RandomForestRegressor(n_estimators=1000,max_depth=10)
regressor.fit(X_train,np.ravel(y_train,order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [23]:
prediction=regressor.predict(X_test)

In [24]:
r2_score(y_test,prediction)

0.6791638252843117

In [25]:
file='model.sav'
pickle.dump(regressor,open(file,'wb'))